In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (18.39 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|18,385,082|14|18,385,082|GetValue
|PnLExplainReport:data|1|18,385,067|449|18,385,067|GetValue/Calc
|TradingContainer:NPV|11|18,325,874|178|1,665,988|GetValue
|TradingContainer:NPV|11|18,325,695|98|1,665,972|GetValue/Calc
|Portfolio:items|121|18,028,629|1,271|148,996|GetValue
|Portfolio:items|121|18,028,524|937|148,996|GetValue/Calc
|Workbook:items|1,104|17,134,644|11,818|15,520|GetValue
|Workbook:items|1,100|17,122,825|8,744|15,566|GetValue/Calc
|PnLExplainReport:cutoffs|1|9,792,665|23|9,792,665|GetValue
|PnLExplainReport:cutoffs|1|9,792,642|217|9,792,642|GetValue/Calc
|Root:Clocks|548|9,792,293|1,928,434|17,869|Context
|TradeOpenEvent|1,014|3,797,460|3,797,460|3,745|Db.Get
|TradeOpenEvent:ticket|22,286|3,539,755|122,991|158|GetValue
|TradingTicket|1,014|3,416,764|3,416,764|3,369|Db.Get
|Root:End|278|1,088,122|1,013,856|3,914|Context
|Root:Activity MarketData|255|1,087,496|1,006,866|4,264|Context
|Root:Amend Portfolio|255|1,062,170|981,146|4,165|Context
|Root:Start|270|1,034,288|952,167|3,830|Context
|Root:Start breaks|255|1,033,781|954,622|4,054|Context
|Root:Amend MarketData|255|868,766|791,948|3,406|Context
|Portfolio:children|242|852,573|1,698|3,523|GetValue
|Portfolio:children|121|850,875|967|7,032|GetValue/Calc
|Root:Amend Trading|255|829,155|754,827|3,251|Context
|Root:Activity Portfolio|263|795,819|725,435|3,025|Context
|Root:Activity Trading|263|788,221|716,074|2,997|Context
|PortfolioUpdateEvent:children|121|345,086|2,634|2,851|GetValue
|TradingBook|102|317,126|317,126|3,109|Db.Get
|Equity:NPV|15|294,674|187|19,644|GetValue
|Equity:NPV|15|294,486|106|19,632|GetValue/Calc
|MarketInterface:spot|15|140,969|154|9,397|GetValue
|MarketInterface:spot|15|140,815|106|9,387|GetValue/Calc
|ExternalRefData:state|15|133,441|148|8,896|GetValue
|ExternalRefData:state|15|133,293|105|8,886|GetValue/Calc
|RefData:state|15|132,807|154|8,853|GetValue
|RefData:state|15|132,653|113|8,843|GetValue/Calc
|_WorkItemEvent:book1|22,286|113,527|113,527|5|GetValue
|_WorkItemEvent:book2|22,286|112,623|107,077|5|GetValue
|TradeOpenEvent:quantity|22,286|102,282|102,282|4|GetValue
|Clock:cutoffs|2,480|90,849|10,059|36|GetValue
|_WorkItemEvent:item|11,143|81,327|75,663|7|GetValue
|Clock:cutoffs|20|80,865|141|4,043|GetValue/Calc
|Clock:parent|20|79,919|184|3,995|GetValue
|Clock:parent|20|79,735|157|3,986|GetValue/Calc
|TradeOpenEvent:action|11,143|75,986|75,986|6|GetValue
|TradeOpenEvent:premium|11,143|72,350|69,509|6|GetValue
|TradeOpenEvent:unitPrice|11,143|62,779|62,779|5|GetValue
|Event:amends|11,340|60,524|60,524|5|GetValue
|TradingBook:clock|2,200|53,431|20,358|24|GetValue
|PortfolioUpdateEvent|12|44,818|44,818|3,734|Db.Get
|RefDataUpdateEvent|9|34,957|34,957|3,884|Db.Get
|TradingBook:clock|1,100|33,072|11,452|30|GetValue/Calc
|TradingPortfolio|10|30,870|30,870|3,087|Db.Get
|Clock|5|14,894|14,894|2,978|Db.Get
|Portfolio:clock|242|9,159|2,373|37|GetValue
|Portfolio:books|231|8,025|1,808|34|GetValue
|Equity:refdata|15|7,117|170|474|GetValue
|Equity:refdata|15|6,946|116|463|GetValue/Calc
|Portfolio:clock|121|6,785|1,256|56|GetValue/Calc
|MarketInterface:source|15|6,666|136|444|GetValue
|Portfolio:books|121|6,599|885|54|GetValue/Calc
|MarketInterface:source|15|6,529|107|435|GetValue/Calc
|ClockEvent|2|6,376|6,376|3,188|Db.Get
|MarketInterface|2|6,203|6,203|3,101|Db.Get
|ClockEvent:parent|8|5,785|83|723|GetValue
|Equity|2|5,664|5,664|2,832|Db.Get
|MarketDataSource|2|5,656|5,656|2,828|Db.Get
|RootClock|1|4,117|4,117|4,117|Db.Get
|MarketDataSource:clock|30|3,590|288|119|GetValue
|MarketDataSource:clock|15|3,302|150|220|GetValue/Calc
|ForwardCashflow|1|2,840|2,840|2,840|Db.Get
|Entity:clock|40|801|341|20|GetValue
|Entity:clock|20|460|194|23|GetValue/Calc
|ForwardCashflow:NPV|11|348|141|31|GetValue
|MarketInterface:sourceName|15|328|133|21|GetValue
|RootClock:cutoffs|53|314|205|5|GetValue
|RefDataUpdateEvent:data|58|292|292|5|GetValue
|ForwardCashflow:NPV|11|207|90|18|GetValue/Calc
|MarketInterface:sourceName|15|194|108|12|GetValue/Calc
|Equity:assetName|15|109|109|7|GetValue
|RootClock:cutoffs|1|108|50|108|GetValue/Calc
|RootClock:cosmicAll|1|36|15|36|GetValue
|CosmicAll:dbState|1|21|13|21|GetValue
|RootClock:cosmicAll|1|20|20|20|GetValue/Calc
|PnLExplainReport:valuable|2|9|9|4|GetValue
|CosmicAll:dbState|1|8|8|8|GetValue/Calc
|PnLExplainReport:ts2|2|8|8|4|GetValue
|PnLExplainReport:ts1|2|7|7|3|GetValue

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

In [5]:
dba = db.copy()
valuable = dba.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with SummaryMonitor():
    with Context({clock.cutoffs: ts7}):
        print valuable.NPV()
        node = getNode(valuable.NPV)
node.printInputGraph()

40.85


# Compute activity summary (0.15 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|35
|GetValue|185
|GetValue/Calc|24

<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4473028504 @4561426320 T=False>, nIn=3
   <Node: /Global/Equity/a716a276-736b-4e5a-a96c-473d96a17c48.(Equity:NPV)() in Root:4473028504 @4632152464 T=False>, nIn=2
      <Node: /Global/Equity/a716a276-736b-4e5a-a96c-473d96a17c48.(Equity:refdata)() in Root:4473028504 @4632152592 T=False>, nIn=1
         <Node: /Global/Equity/a716a276-736b-4e5a-a96c-473d96a17c48.(Equity:assetName)() in Root:4473028504 @4632152720 T=False>, nIn=0
      <Node: /Global/MarketInterface/GOOG.(MarketInterface:spot)() in Root:4473028504 @4589534672 T=True>, nIn=2
         <Node: /Global/MarketDataSource/source1.GOOG.(ExternalRefData:state)() in Root:4473028504 @4561424848 T=False>, nIn=1
            <Node: /Global/MarketDataSource/source1.GOOG.(RefData:state)() in Root:4473028504 @4561428048 T=False>, nIn=10
               <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:4473028504 @4653642512 T=True>, nIn=2
                  <Node: /Glo

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [6]:
from mand.graph import DependencyManager, setDependencyManager
from mand.core import Event

class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items')
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)


    def calculated(self, node):
        if not node.isSimplified:
            return
        ok = True
        for input in node.tweakPoints():
            if input not in node.ctx.tweaks:
                print 'context simplification failure', '%s used %s' % (str(node), str(input))
                addFootnote(text='context simplification failure', info='%s used %s' % (str(node.key), str(input.key)))
                ok = False
        return ok
                  
    def getCtx(self, clock):
        ts = clock.cutoffs()
        cKey = (clock, ts) # really want value-based comparison of ts, not object equality
        if cKey not in self.contexts:
            ctx = Context({clock.cutoffs: ts})
            self.contexts[cKey] = ctx
            print 'new simp-ctx:', ctx.name, clock
        return self.contexts[cKey]
            
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        if key.fullName() in self.simpleClockMethods:
            obj = key.object()
            ctx1 = self.getCtx(obj.clock())
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
setDependencyManager(DM1())

db4 = db.copy()
valuable = db4.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with Context({clock.cutoffs: ts7}):
    print valuable.NPV()
    node = getNode(valuable.NPV)
    node.printInputGraph()

new simp-ctx: Root:4473028504:4473028504 <Entity:/Global/Clock/Trading>
new simp-ctx: Root:4473028504:4473028504 <Entity:/Global/Clock/MarketData>
40.85
 <Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4473028504 @4800109008 T=False> *not evaluated*
   <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:4473028504 @4799958480 T=True> *not evaluated*
     <Node: /Global/Clock/Trading.(Clock:parent)() in Root:4473028504 @4800034640 T=False> *not evaluated*
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4473028504 @4800109456 T=True>, nIn=0
     <Node: /Global/Clock/Workflow.(Clock:cutoffs)() in Root:4473028504 @4717219024 T=True> *not evaluated*
       <Node: /Global/Clock/Workflow.(Clock:parent)() in Root:4473028504 @4717218896 T=False> *not evaluated*
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4473028504 @4800109456 T=True>, nIn=0
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4473028504 @480

In [7]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

new simp-ctx: Root:Clocks:4473028504 <Entity:/Global/Clock/Portfolio>
new simp-ctx: Root:Clocks:4473028504:4473028504 <Entity:/Global/Clock/Trading>
context simplification failure <Node: /Global/TradingPortfolio/Eq-Prop.(Portfolio:items)() in Root:Clocks:4473028504 @4530238480 T=False> used <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:Clocks:4473028504 @4574484368 T=True>
context simplification failure <Node: /Global/TradingPortfolio/Eq-Prop.(Portfolio:items)() in Root:Clocks:4473028504 @4530238480 T=False> used <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:Clocks:4473028504:4473028504 @4568946320 T=True>
context simplification failure <Node: /Global/TradingPortfolio/Eq-Inst.(Portfolio:items)() in Root:Clocks:4473028504 @4580334032 T=False> used <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:Clocks:4473028504 @4574484368 T=True>
context simplification failure <Node: /Global/TradingPortfolio/Eq-Inst.(Portfolio:items)() in Root:Clocks:4473028504 @4580334032 T=F

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|0.00
|activity: MarketData|5240.00
|activity: Portfolio|1016.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. context simplification failure
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (15.47 seconds of wall clock time)

|key|value|
|-|-|
|Context|19
|Db.Get|2176
|GetValue|43096
|GetValue/Calc|8869


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|15,465,648|17|15,465,648|GetValue
|PnLExplainReport:data|1|15,465,631|605|15,465,631|GetValue/Calc
|TradingContainer:NPV|11|15,458,139|150|1,405,285|GetValue
|TradingContainer:NPV|11|15,457,989|116|1,405,271|GetValue/Calc
|Portfolio:items|121|15,316,508|49,000|126,582|GetValue
|Portfolio:items|121|15,311,249|1,086|126,539|GetValue/Calc
|Root:Clocks:4473028504|7,609|15,259,752|5,598,401|2,005|Context
|PnLExplainReport:cutoffs|1|11,074,004|14|11,074,004|GetValue
|PnLExplainReport:cutoffs|1|11,073,990|136|11,073,990|GetValue/Calc
|Root:Clocks|61|11,073,736|32,751|181,536|Context
|Workbook:items|1,104|9,043,198|6,278|8,191|GetValue
|Workbook:items|200|9,036,920|1,630|45,184|GetValue/Calc
|Root:Clocks:4473028504:4473028504|600|9,035,289|1,874,874|15,058|Context
|Clock:cutoffs|2,477|5,658,936|18,053|2,284|GetValue
|Clock:cutoffs|1,624|5,648,282|11,392|3,478|GetValue/Calc
|Clock:parent|1,624|5,583,270|14,758|3,437|GetValue
|Clock:parent|1,624|5,568,511|11,748|3,428|GetValue/Calc
|TradeOpenEvent|1,014|3,711,814|3,711,814|3,660|Db.Get
|TradeOpenEvent:ticket|4,052|3,292,103|30,427|812|GetValue
|TradingTicket|1,014|3,261,676|3,261,676|3,216|Db.Get
|Root:Activity Portfolio|18|739,541|773|41,085|Context
|Root:Start|23|718,598|13,625|31,243|Context
|Root:Start breaks|11|717,371|605|65,215|Context
|Root:End|38|714,059|17,377|18,791|Context
|Root:Amend MarketData|11|702,010|850|63,819|Context
|Root:Activity Trading|18|697,157|989|38,730|Context
|Portfolio:children|352|512,293|1,889|1,455|GetValue
|Portfolio:children|33|510,403|269|15,466|GetValue/Calc
|PortfolioUpdateEvent:children|33|340,728|1,356|10,325|GetValue
|TradingBook|102|315,047|315,047|3,088|Db.Get
|Equity:NPV|15|117,361|222|7,824|GetValue
|Equity:NPV|15|117,139|121|7,809|GetValue/Calc
|MarketInterface:spot|15|109,011|167|7,267|GetValue
|MarketInterface:spot|15|108,844|124|7,256|GetValue/Calc
|Entity:clock|3,248|104,728|40,067|32|GetValue
|ExternalRefData:state|15|101,119|174|6,741|GetValue
|ExternalRefData:state|15|100,944|128|6,729|GetValue/Calc
|Root:Amend Portfolio:4473028504|450|94,356|67,027|209|Context
|Entity:clock|3,248|64,661|28,252|19|GetValue/Calc
|Root:Amend Portfolio|11|60,443|845|5,494|Context
|TradingBook:clock|1,504|50,714|17,667|33|GetValue
|RefData:state|15|46,913|98|3,127|GetValue
|RefData:state|4|46,814|32|11,703|GetValue/Calc
|PortfolioUpdateEvent|12|44,022|44,022|3,668|Db.Get
|TradingBook:clock|1,504|33,047|12,323|21|GetValue/Calc
|RefDataUpdateEvent|9|31,813|31,813|3,534|Db.Get
|TradingPortfolio|10|30,790|30,790|3,079|Db.Get
|_WorkItemEvent:book2|4,052|27,190|20,724|6|GetValue
|_WorkItemEvent:item|2,026|23,477|17,664|11|GetValue
|_WorkItemEvent:book1|4,052|22,261|22,261|5|GetValue
|Root:Amend Trading|11|20,558|609|1,868|Context
|TradeOpenEvent:quantity|4,052|19,615|19,615|4|GetValue
|Root:Activity MarketData|11|17,589|559|1,599|Context
|TradeOpenEvent:premium|2,026|16,663|13,688|8|GetValue
|RootClock:cutoffs|4,063|15,653|15,560|3|GetValue
|Portfolio:books|231|15,207|2,909|65|GetValue
|Clock|5|14,925|14,925|2,985|Db.Get
|TradeOpenEvent:action|2,026|14,804|14,804|7|GetValue
|Event:amends|2,887|14,095|14,095|4|GetValue
|Portfolio:books|231|13,707|1,921|59|GetValue/Calc
|TradeOpenEvent:unitPrice|2,026|11,827|11,827|5|GetValue
|ClockEvent:parent|810|9,070|3,332|11|GetValue
|Portfolio:clock|140|8,548|1,898|61|GetValue
|Equity:refdata|15|7,074|193|471|GetValue
|Equity:refdata|15|6,881|137|458|GetValue/Calc
|MarketInterface:source|15|6,841|165|456|GetValue
|MarketInterface:source|15|6,675|117|445|GetValue/Calc
|Portfolio:clock|140|6,649|1,271|47|GetValue/Calc
|ClockEvent|2|6,546|6,546|3,273|Db.Get
|MarketInterface|2|5,971|5,971|2,985|Db.Get
|Equity|2|5,813|5,813|2,906|Db.Get
|MarketDataSource|2|5,604|5,604|2,802|Db.Get
|Root:Amend MarketData:4473028504|3|3,823|3,742|1,274|Context
|RootClock|1|3,669|3,669|3,669|Db.Get
|MarketDataSource:clock|23|3,528|246|153|GetValue
|MarketDataSource:clock|23|3,282|184|142|GetValue/Calc
|ForwardCashflow|1|2,974|2,974|2,974|Db.Get
|ForwardCashflow:NPV|11|504|242|45|GetValue
|MarketInterface:sourceName|15|395|180|26|GetValue
|ForwardCashflow:NPV|11|261|127|23|GetValue/Calc
|MarketInterface:sourceName|15|215|129|14|GetValue/Calc
|Equity:assetName|15|129|129|8|GetValue
|RootClock:cutoffs|1|92|46|92|GetValue/Calc
|RefDataUpdateEvent:data|15|72|72|4|GetValue
|CosmicAll:dbState|1|27|22|27|GetValue
|RootClock:cosmicAll|1|18|10|18|GetValue
|PnLExplainReport:valuable|2|14|14|7|GetValue
|PnLExplainReport:ts1|2|11|11|5|GetValue
|PnLExplainReport:ts2|2|10|10|5|GetValue
|RootClock:cosmicAll|1|8|8|8|GetValue/Calc
|CosmicAll:dbState|1|5|5|5|GetValue/Calc